In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import date, timedelta
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

BDO

In [ ]:
STOCK_CODE='MAC'

In [ ]:
end = date.today()
start = end - timedelta(days = 4)

In [ ]:
# Get updated value from phisix-api
latest = requests.get(f'http://phisix-api.appspot.com/stocks/{STOCK_CODE}.json').json()
last_price = latest['stock'][0]['price']['amount']
print(last_price)

# get previos data from pselookup
data = requests.get(f'https://pselookup.vrymel.com/api/stocks/{STOCK_CODE}/history/{start}/{end}', timeout=5).json()['history']
before = data[-1]['close']
data[-1]['close']=last_price
after = data[-1]['close']
print(f'before:{before} after:{after}')

In [ ]:
# df = pd.json_normalize(data)
# layout = dict(
#             title=f"{STOCK_CODE} Stock Data",
#             xaxis = dict(
#             type="category", 
#             categoryorder='category ascending'))
# fig = go.Figure(data=[go.Candlestick(x=df['trading_date'],
#                 open=df['open'],
#                 high=df['high'],
#                 low=df['low'],
#                 close=df['close'])], layout=layout)
# fig.show()

In [ ]:
oo_indexed = df.set_index('trading_date')
# oo_indexed.volume.plot(kind='bar',figsize=(20,10))

In [ ]:
# oo_indexed.head(5)

In [ ]:
for i in range(2,oo_indexed.shape[0]):
    current = oo_indexed.iloc[i,:]
    prev = oo_indexed.iloc[i-1,:]
    prev_2 = oo_indexed.iloc[i-2,:]

    realbody = abs(current['open'] - current['close'])
    candle_range = current['high'] - current['low']


    if current['open'] < current['close']:
        candle_type='Bull'
        head_wick = current['high'] - current['close']
        tail_wick = current['open'] - current['low']
    elif current['open'] == current['close']: 
        oo_indexed.loc[idx, 'Doji'] = True
    else: 
        candle_type='Bear'
        head_wick = current['high'] - current['open']
        tail_wick = current['close'] - current['low']

    idx = oo_indexed.index[i]

    # Bullish swing
    oo_indexed.loc[idx,'Bullish swing'] = current['low'] > prev['low'] and prev['low'] < prev_2['low']

    # Bearish swing
    oo_indexed.loc[idx,'Bearish swing'] = current['high'] < prev['high'] and prev['high'] > prev_2['high']

    # Bullish pinbar
    oo_indexed.loc[idx,'Bullish pinbar'] = realbody <= candle_range/3 and  min(current['open'], current['close']) > (current['high'] + current['low'])/2 and current['low'] < prev['low']

    # Bearish pinbar
    oo_indexed.loc[idx,'Bearish pinbar'] = realbody <= candle_range/3 and max(current['open'] , current['close']) < (current['high'] + current['low'])/2 and current['high'] > prev['high']

    # Inside bar
    oo_indexed.loc[idx,'Inside bar'] = current['high'] < prev['high'] and current['low'] > prev['low']

    # Outside bar
    oo_indexed.loc[idx,'Outside bar'] = current['high'] > prev['high'] and current['low'] < prev['low']

    # Bullish engulfing
    oo_indexed.loc[idx,'Bullish engulfing'] = current['high'] > prev['high'] and current['low'] < prev['low'] and realbody >= 0.8 * candle_range and current['close'] > current['open']

    # Bearish engulfing
    oo_indexed.loc[idx,'Bearish engulfing'] = current['high'] > prev['high'] and current['low'] < prev['low'] and realbody >= 0.8 * candle_range and current['close'] < current['open']

    # Bearish Hammer
    if realbody > head_wick*2 and tail_wick > head_wick*2:
        oo_indexed.loc[idx,f'{candle_type}ish hammer'] = realbody*2 < tail_wick
    else:
        oo_indexed.loc[idx,f'{candle_type}ish hammer'] = False


In [ ]:
bull_swing = oo_indexed[oo_indexed['Bullish swing'] == True]
bull_swing

In [ ]:
bull_pinbar = oo_indexed[oo_indexed['Bullish pinbar'] == True]
bull_pinbar

In [ ]:
outside_bar = oo_indexed[oo_indexed['Outside bar'] == True]
outside_bar

In [ ]:
bull_engulfing = oo_indexed[oo_indexed['Bullish engulfing'] == True]
bull_engulfing

In [ ]:
bear_swing = oo_indexed[oo_indexed['Bearish swing'] == True]
bear_swing

In [ ]:
bear_pinbar = oo_indexed[oo_indexed['Bearish pinbar'] == True]
bear_pinbar

In [ ]:
inside_bar = oo_indexed[oo_indexed['Inside bar'] == True]
inside_bar

In [ ]:
bear_engulfing = oo_indexed[oo_indexed['Bearish engulfing'] == True]
bear_engulfing

In [ ]:
bull_hammer = oo_indexed[oo_indexed['Bullish hammer'] == True]
bull_hammer

In [ ]:
bear_hammer = oo_indexed[oo_indexed['Bearish hammer'] == True]
bear_hammer